# Improvement towards the reflexion computation from NoiseModelling 

<br><br>

---
### Imports

In [ ]:
import sys
sys.path += [".."] # os.path.abspath("../img-phy-sim")

import img_phy_sim as ips
from img_phy_sim.data import PhysGenDataset, get_image

import os
import random

import numpy as np
import cv2
from torch.utils.data import DataLoader

---
### Data Loading

In [ ]:
def numpy_info_ORIGINAL(numpy_array, should_print=True):
    result = "Array Statistics:"
    result += f"\n    - mean = {numpy_array.mean():.2f}"
    result += f"\n    - median = {np.median(numpy_array):.2f}"
    # result += f"\n    - percentile = {np.percentile(numpy_array, [25, 50, 75])}"
    result += f"\n    - std = {numpy_array.std():.2f}"
    result += f"\n    - var = {numpy_array.var():.2f}"
    result += f"\n    - min = {numpy_array.min():.2f}"
    result += f"\n    - max = {numpy_array.max():.2f}"
    result += f"\n    - shape = {numpy_array.shape}"
    
    if should_print:
        print(result)
    return result

In [ ]:
data_path = "../datasets/physgen_train_raw/osm/"

nm_gt_file = data_path + random.sample(os.listdir(data_path), 1)[0]
# img_src = data_path + "input_physgen_2822.png"
should_scale = True
nm_gt_file

In [ ]:
nm_gt = cv2.imread(nm_gt_file, cv2.IMREAD_GRAYSCALE)
nm_gt.shape

Or:

In [ ]:
# img_src = "./cache_img.png"
# ips.img.save(
#     get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="complex_only", 
#               shuffle=True, return_output=False, as_numpy_array=True), 
#     img_src, should_scale=False)
# should_scale = False

# img_src

In [ ]:
nm_in, nm_gt = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="complex_only", 
               shuffle=False, return_output=True, as_numpy_array=True)

nm_in_file = "./cache_img_1_input.png"
ips.img.save(nm_in, nm_in_file, should_scale=False)

nm_gt_file = "./cache_img_1_output.png"
ips.img.save(nm_gt, nm_gt_file, should_scale=False)

In [ ]:
ips.math.numpy_info(nm_gt);

In [ ]:
ips.img.imshow(nm_gt, size=4)

nm_gt_test = nm_gt / 255
nm_gt_test = nm_gt_test != 0.0

ips.img.imshow(nm_gt_test, size=4)

---
### Error Measurement Definition

In [ ]:
def calc_metrices_ORIGINAL(rays, noise_modelling_gt, rays_format_is_image=False, eval_name="", should_print=True):
    # Create image from rays
    if rays_format_is_image:
        ray_img = rays
    else:
        ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True, 
                                            output_format="single_image", 
                                            img_background=None, ray_value=1.0, ray_thickness=1, 
                                            img_shape=(256, 256), dtype=float, standard_value=0,
                                            should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
    
    # Normalize both (if needed)
    if (noise_modelling_gt > 1.0).any():
        # raise ValueError("Noise Modelling Ground Truth Image is not normalized.")
        noise_modelling_gt /= 255

    if (ray_img > 1.0).any():
        # raise ValueError("Ray Image is not normalized.")
        ray_img /= 255

    # Thresholding to binary images
    noise_modelling_gt_binary = noise_modelling_gt != 0.0
    # numpy_info(noise_modelling_gt_binary)
    rays_binary = ray_img != 0.0

    # Recall, Precision, F1 Score
    overlap = noise_modelling_gt_binary * rays_binary

    #     recall - how is the coverage towards the gt?
    recall = np.sum(overlap) / np.sum(noise_modelling_gt_binary)

    #     precision - how many rays hit the right place?
    precision = np.sum(overlap) / np.sum(rays_binary)

    #     f1
    f1 = 2*(precision*recall) / (precision+recall)

    if should_print:
        print(f"Eval {eval_name}: F1={f1:.02f}, Recall={recall:.02f}, Precision={precision:.02f}")
    return f1, recall, precision

---
### Error Measurement Testing

In [ ]:
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=nm_in, 
                                   directions_in_degree=ips.math.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=None, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)
ips.ray_tracing.print_rays_info(rays)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=nm_in, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=1, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img!=0.0, size=4)

In [ ]:
f1, recall, precision = ips.eval.calc_metrices(rays, nm_gt)

In [ ]:
f1, recall, precision = ips.eval.calc_metrices(nm_gt, nm_gt, rays_format_is_image=True)

---
### Ray-Amount Test

In [ ]:
for i in [100, 10, 1, 0.1]:
    rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                        img_src=nm_in, 
                                        directions_in_degree=ips.math.get_linear_degree_range(start=0, stop=360, step_size=i),
                                        wall_values=None, 
                                        wall_thickness=0,
                                        img_border_also_collide=False,
                                        reflexion_order=3,
                                        should_scale_rays=True,
                                        should_scale_img=False)
    f1, recall, precision = ips.eval.calc_metrices(rays, nm_gt, eval_name=f"{len(ips.math.get_linear_degree_range(start=0, stop=360, step_size=i))} Rays")

f1, recall, precision = ips.eval.calc_metrices(nm_gt, nm_gt, rays_format_is_image=True, eval_name="Ground Truth")

---
### Baseline

All experiments are based now on 36 rays.

In [ ]:
dataset = PhysGenDataset(mode='test', variation="sound_reflection", input_type="osm", output_type="complex_only")

f1_mean = 0
recall_mean = 0
precision_mean = 0
counter = 0
for (input_img, target_img, idx) in dataset:
    rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                        img_src=input_img.squeeze(0).numpy(), 
                                        directions_in_degree=ips.math.get_linear_degree_range(start=0, stop=360, step_size=36),
                                        wall_values=None, 
                                        wall_thickness=0,
                                        img_border_also_collide=False,
                                        reflexion_order=3,
                                        should_scale_rays=True,
                                        should_scale_img=False)
    f1, recall, precision = ips.eval.calc_metrices(rays, nm_gt, eval_name=f"{len(ips.math.get_linear_degree_range(start=0, stop=360, step_size=i))} Rays", should_print=False)
    f1_mean += f1
    recall_mean += recall
    precision_mean += precision
    counter += 1

f1_mean /= counter
recall_mean /= counter
precision_mean /= counter

print(f"Baseline Accuracy: F1={f1_mean:.2f}, Recall={recall_mean:.02f}, Precision={precision_mean:.02f}")

---
### ISM

In [ ]:
dataset = PhysGenDataset(mode='test', variation="sound_reflection", input_type="osm", output_type="complex_only")

f1_mean = 0
recall_mean = 0
precision_mean = 0
counter = 0
for (input_img, target_img, idx) in dataset:
    reflection_map, _ = ips.ism.compute_reflection_map(
        source_rel=(0.5, 0.5),
        img=input_img.squeeze(0).numpy(),
        wall_values=[0],   
        wall_thickness=1,
        max_order=1,
        step_px=1,
        parallelization=-1
    )
    reflection_map = ips.ism.reflection_map_to_img(reflection_map)
    
    f1, recall, precision = ips.eval.calc_metrices(reflection_map, nm_gt, rays_format_is_image=True, eval_name=f"{len(ips.math.get_linear_degree_range(start=0, stop=360, step_size=i))} Rays", should_print=False)
    f1_mean += f1
    recall_mean += recall
    precision_mean += precision
    counter += 1

f1_mean /= counter
recall_mean /= counter
precision_mean /= counter

print(f"Baseline Accuracy: F1={f1_mean:.2f}, Recall={recall_mean:.02f}, Precision={precision_mean:.02f}")